In [1]:
import numpy as np
import pandas as pd
import os, errno

def silentremove(filename):
   try:
       os.remove(filename)
   except OSError as e: # this would be "except OSError, e:" before Python 2.6
       if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
           raise # re-raise exception if a different error occurred

# Setup
base = '../'

#########################
## Parent Company File ##
#########################
# Attach to the HDF5 file where we will store the data rolled-up to the parent level
parent_filename = base + 'Data/ReferenceUSA/parent.h5'
# Delete the file that's already in this location in order to ensure that we're not appending data twice
silentremove(parent_filename)
# Setup the connection to the parent file
parent = pd.HDFStore(parent_filename)


########################
## Get Reference Data ##
########################
# Create a filename where we can store all records in an HDF5 file
clean_filename = base + 'Data/ReferenceUSA/sample_clean.h5'
# Attach to the HDF5 file with the clean referenceUSA data
store = pd.HDFStore(clean_filename)
# Get the sturcture data that will identify parents, subsidiaries, branches, and independent establishments
structure = store.select('structure')
# Get the main data to identify companies
main = store.select('main')
# Join the main and structure dataframes for processing
main = main.join(structure)
# Delete the structure dataframe to free up memory
del structure 

In [2]:
#########################################
## Identify Firm Groups and Head Firms ##
#########################################
# Find all the firms where the ABI is the same as the Parent number. This identifies the vast majority of the head firms
main['main_head'] = 0
main.loc[(main.ABI == main.Parent_Number), 'main_head'] = 1

# Find the firm groups, where a group is all companies under a given parent number in a given year
grouped = main.groupby(['archive_version_year', 'Parent_Number'], as_index=False, sort=True)
# Get the group_id for each observation so we can map them to groups
group_id = grouped.grouper.group_info[0]
# Add this group id to the main dataframe
main['group_id'] = group_id

#Find out how many head firms are in each firm group. It should be one, but some don't have any
test = main.groupby(['group_id'], sort=True, as_index=False).sum()
# Create a dataframe of the firm groups with no head firms
group_miss = test[test.main_head == 0]['group_id']
# Merge the list of groups with a missing head onto the main dataframe so we have an indicator for headless groups
# <- need to find a way to make this indicator a binary variable instead of this category thing that ends up 
# <- being a string
main = main.merge(pd.DataFrame(group_miss), how='left', on='group_id', indicator = True)
# Create a dataframe of all the firms in firm groups with no head firm
drop = pd.DataFrame(main[main._merge == 'both']['_merge'])
# Delete the indicator for groups missing head firms
del main['_merge']
del test
del group_miss

In [3]:
###########################################
## Reidentify Firm Groups and Head Firms ##
###########################################
# Merge in the drop list so we know which firms to drop from the main list 
main = main.join(drop, how='left')

# Drop these firms
main = main[main._merge != 'both']

# Redo the head indicator to make sure ever firm group has exactly one head firm
main['main_head'] = 0
main.loc[(main.ABI == main.Parent_Number), 'main_head'] = 1

# Store the index of head firms in a main_head dataframe for future matching
main_head = main['main_head']


# Create a new grouping now that some firm groups have been removed
grouped = main.groupby(['archive_version_year', 'Parent_Number'], as_index=False, sort=True)
# Now that we have the grouping, we don't need the main dataframe, so delete this to clear up memory
del main

# Get the group_id for each observation so we can map them to groups
group_id = grouped.grouper.group_info[0]

In [4]:
#######################
## Main Parent Table ##
#######################
# Get the main information from the first observation in each firm group. Since this information is the same for all 
# observations in a group, it doesn't matter which observation it comes from
parent_main = grouped.first()
# Only keep the Parent number (which will be the ABI for the whole group), the archive version, and the company name
parent_main = parent_main[['Parent_Number', 'archive_version_year', 'Company']]
# Rename the Parent number column to reflect the fact that it will be the ABI number for the group
parent_main.columns = ['ABI', 'archive_version_year', 'Company']
# Simplify the datatypes to save space
parent_main['ABI'] = parent_main.ABI.astype('int32')
parent_main['archive_version_year'] = parent_main.archive_version_year.astype('int16')
# Change the index name to group_id
parent_main.index.name = 'group_id'
# Save the main data
parent.append('main', parent_main, index=False, dropna=False, expectedrows = 41528347)
# Delete the main parent data to save memory
del parent_main

In [5]:
#####################
## Structure Table ##
#####################
# Aggregate the columns that we want to use
parent_structure = grouped.agg({'IDCode': 'mean', 'Company_Holding_Status':'max', 'Business_Status_Code':'min'})
# Only keep the columns needed in the structure dataframe
parent_structure = parent_structure[['IDCode', 'Company_Holding_Status', 'Business_Status_Code']]
# Simplify the datatypes to save space
parent_structure['IDCode'] = parent_structure.IDCode.astype('uint8')
parent_structure['Business_Status_Code'] = parent_structure.Business_Status_Code.astype('uint8')
# Change the index name to group_id
parent_structure.index.name = 'group_id'
# Save the structure data
parent.append('structure', parent_structure, index=False, dropna=False, expectedrows = 41528347)
# Going to need to redo the structure data
# Delete the parent structure data to save memory
del parent_structure
# delete the group structures, since we don't need these anymore
del grouped

In [6]:
####################
## Descript Table ##
####################
descript = store.select('descript')
# Merge in the drop list so we know which firms to drop from the list 
descript = descript.join(drop, how='left')
# Drop these firms
descript = descript[descript._merge != 'both']
# Delete the indicator column
del descript['_merge']
# Add in the group_id's to assign observations to groups
descript['group_id'] = group_id
# Group the data using these groups
parent_group = descript.groupby(['group_id'], sort=True)
# Now that we have the grouping, delete the original data to save memory
del descript
# Aggregate everything by sum, except establishment year which will use the min
parent_desc = parent_group.sum()
parent_desc2 = parent_group.min()
del parent_group
parent_desc['year_established'] = parent_desc2['year_established']
# Save the descript data and remove it from the workspace to save memory
del parent_desc2
parent.append('descript', parent_desc, index=False, dropna=False, expectedrows = 41528347)
del parent_desc

In [7]:
####################
## Industry Table ##
####################
industry = store.select('industry')
# Merge in the drop list so we know which firms to drop from the list 
industry = industry.join(drop, how='left')
# Drop these firms
industry = industry[industry._merge != 'both']
# Delete the indicator column
del industry['_merge']
# Add in the group_id's to assign observations to groups
industry['group_id'] = group_id
# Only keep the industry information from the head firm in each group
parent_industry = industry[main_head == 1]
# Set the group_id as the index for the parent industry dataframe
parent_industry.set_index('group_id', inplace=True)

# Save the industry data and remove it from the workspace to save memory
parent.append('industry', parent_industry, index=False, dropna=False, expectedrows = 41528347)
del parent_industry
del industry

In [8]:
####################
## Geo Table ##
####################
geo = store.select('geo')
# Merge in the drop list so we know which firms to drop from the list 
geo = geo.join(drop, how='left')
# Drop these firms
geo = geo[geo._merge != 'both']
# Delete the indicator column
del geo['_merge']
# Add in the group_id's to assign observations to groups
geo['group_id'] = group_id
# Only keep the geo information from the head firm in each group
parent_geo = geo[main_head == 1]
# Set the group_id as the index for the parent industry dataframe
parent_geo.set_index('group_id', inplace=True)

# Save the geo data and remove it from the workspace to save memory
parent.append('geo', parent_geo, index=False, dropna=False, expectedrows = 41528347)
del parent_geo
del geo